<a href="https://colab.research.google.com/github/jhmartel/fp/blob/master/_notebooks/2022-04-20-MillionaireGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yang's Millionaire's Problem. Part 1. 

> " We begin the study of Yang's Millionaire Problem, approaching via convex analysis. Two players $A,B$ have secret points $s_A, s_B \in \mathbf{R}^2$, and the first player to guess an affine function which separates the secrets wins!  "

- toc: false
- branch: master
- badges: false
- comments: true
- categories: [millionaire, secret, convex analysis]
- hide: false




The purpose of this article is to investigate whether there is strategy or skill possible in the following variation of the "Millionaire Problem". [reference].

Here is the game. We have a huge grid $\mathbf{R}^2$. Now let two players $A,B$ have secret locations $s_A=(x_A, y_A)$ and $s_B=(x_B, y_B)$. These secrets are _points_ in the euclidean plane $\mathbf{R}^2$. 

Now the players $A, B$ are going to take turns guessing affine functions (or affine lines in $\mathbf{R}^2$) and the first player to guess an affine function which separates the secrets wins! 

The gameplay is something like this: The players $A,B$ take turns. If player $A$ goes first, then player $A$ chooses an affine function $\ell$ on $\mathbf{R}^2$, and asks player $B$ to reply with the sign of $\ell(s_B)$. We require that $B$ replies honestly with $sgn(\ell(s_B))$. This is the end of player $A$'s turn. If $\ell$ separates the secrets, then player A wins. Otherwise it's player B's turn. Next player $B$ chooses an affine function $\ell'$, and asks player $A$ to reply with the sign of $\ell'(s_A)$. Once player $A$ replies, then this is the end of player $B$'s turn. Again, if $\ell'$ separates the secrets, then player B wins. Otherwise it's player A's turn.

The object of the game is to determine an affine function $\ell$ which separates the secrets, i.e. for which $ sgn(\ell(s_A)) \neq sgn(\ell(s_B)).$ The first player to demonstrate an affine function which separates the secrets wins!

Our interest is to find optimal strategies for this game. Firstly, we have to consider whether there is _any_ strategy possible. For example, can player $A$ use the cumulative history of both player $A$ and $B$'s affine guesses to better inform their next guess? For example, if player $A$ guesses an affine function $\ell$ which does not separate, then player $B$ can use the knowledge of their own private secret to determine which halfspace contains $s_A$. And indeed, by the same reasoning player $A$ can use their knowledge of $s_A$ to likewise determine which halfspace contains $s_B$. So obviously the initial distribution $d\lambda$ is updated to the restricted distribution $d\lambda\cdot 1_H$, where $H$ is the halfspace defined by $\ell$ and containing $s_A, s_B$. With successive guesses, the distribution becomes a descending chain of closed convex sets, namely the intersection of successive halfspaces, having the form $$d\lambda \leadsto d\lambda \cdot 1_H \leadsto d\lambda \cdot 1_H 1_{H'} \leadsto d\lambda \cdot 1_H 1_{H'} 1_{H''} \leadsto \cdots. $$

The notation is somewhat strange, but simply expresses that we remain uncertain of the specific location of the secrets $s_A, s_B$, except we know the possibly location is becoming more restricted. 


In the millionaire game, the players $A,B$ have an interest in _privacy_. Their secrets $s_A, s_B$ are intended to be _secret_. This means the players $A,B$ might not choose affine functions which potentially reveal information about their own secrets. In practice this means players determined to maintain their privacy will always choose affine functions which do not bound compact convex sets. Similarly, an opponent will not readily choose affine functions which separates the domain into a bounded component, since the probability that the opponent's secret lies in the bounded component is relatively small, while the probability of its lying in the unbounded component is much greater. 

The subject of so-called _zero knowledge proofs_ in cryptography is related to the millionaires problem. Here we try to find a balance where the players can choose to reveal as much as they wish of their own balances, while their own guesses are signals/indications in-themselves of the secret balance.

Our question is whether there is any strategy or skill in this game. What is the optimal strategy? Can the player use the knowledge of the opponent's affine functions to improve their own selection of affine function??

In [ ]:
import numpy as np

# Now we simulate the millionaire problem on the euclidean two-dimensional plane.
# For convenience we rename the players $A,B$ as players $+1, -1$, respectively. 
# for testing purposes we suppose the players A,B have secrets below:

#s_A=input("What is player A's secret position?")
s_A=[1,0]
s_A=np.array(s_A)

#s_B=input("What is player B's secret position?")
s_B=[0, 0.2]
s_B=np.array(s_B)


# now we define some basic functions, i.e. to compute affine functions based on
# their normal n and height b.
def affine(n,x,b):
    n=np.array(n)
    x=np.array(x)
   # return n.dot(x)+b
    return n[0]*x[0]+n[1]*x[1] + b


# to protect the secret we really only need the sign of the affine function.
def sign(x_Real):
    if x_Real<0: 
        return -1
    else:
        return +1

# here t defines the test function, which is True iff the affine function 
# separates the secrets. 
def t(n,b):
    n=np.array(n)
    if sign(affine(n,s_A,b)) != sign(affine(n,s_B, b)):
         return True
    else:
         return False


In [ ]:
# Now we setup the basic routine, i.e. sequence of gameplay.

outcome=False
history=[]
vector_history=[]
player=+1
i=0
color=[]


while outcome == False:
    print("\n Player " + str(player) + "'s turn to play:" )
    print("Given the history " + str(history) + " choose your affine function:")     
    
    n0 = float(input())
    n1 = float(input())
    b = float(input())
    history = history + [[n0, n1, b]]
    vector_history = vector_history+[[n0,n1]] 
    i=i+1


    if t([n0, n1], b) == True:
        outcome = True
        print("Winner! Player " + str(player)+ " has separated the secrets with " + str([n0, n1, b]) + ".")
    else:
        print("Fail! Player " + str(player) + " has not separated the secrets.")
        player=player*(-1)

    

# the following plots the various normals chosen by the players, but we would
# prefer to have the half spaces. 
    V=np.array(vector_history)
    origin=np.array([[0]*i, [0]*i])
    plt.quiver(*origin, V[:,0], V[:,1], scale=21)
    plt.show()          
